In [ ]:
import os
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
import cv2

data 폴더를 새로 생성하여 x, y의 이름을 가지는 하위폴더를 생성합니다.
기존 train_img파일은 `./data/x/`폴더로 이동합니다.
```bash
mv ./train_img/* ./data/x/
```

따라서 주어진 csv의 img_path를 위 경로에 맞게 변경해야합니다.

In [ ]:
train_csv = pd.read_csv('./train.csv')
train_csv['img_path'] = train_csv['img_path'].apply(lambda x: x.replace('./train_img/', './data/x/'))
train_csv.to_csv('./train.csv', index=False)

`./data/y/` 에는 아래의 코드로 마스크가 디코딩되어 이미지로 저장됩니다.

### 인코딩 된 마스크를 디코딩하여 이미지파일로 저장 (y)

In [ ]:
def rle_decode(mask_rle, shape=(1024, 1024)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1

    img.reshape(shape)
    return img.reshape(shape)

for index, row in tqdm(train_csv.iterrows()):
    image_path = row['img_path']
    mask_rle = row['mask_rle']
    
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    mask = rle_decode(mask_rle)
    
    cv2.imwrite(f'./data/y/{row["img_id"]}.png', mask)


## Image Tiling (256) code
train데이터는 1024 * 1024, test데이터는 224 * 224 입니다. 이에 착안하여 데이터를 더욱 다양화하고 증폭시키자 train이미지를 256사이즈로 tiling하여 별도의 이미지로 저장합니다.

In [ ]:
file_list=glob.glob('./train_img/x/*.png')

output_folder='./data/x/'

image_width = 1024
image_height = 1024

window_size = 256

num_windows_x = (image_width - window_size) // window_size + 1
num_windows_y = (image_height - window_size) // window_size + 1

image_patches = []
for cnt,file in enumerate(file_list):
    image=cv2.imread(file)
    for i in range(num_windows_x):
        for j in range(num_windows_y):
            start_x = i * window_size
            start_y = j * window_size
            end_x = start_x + window_size
            end_y = start_y + window_size

            image_patch = image[start_y:end_y, start_x:end_x]
            image_patches.append(image_patch)

cnt2=0
for i,file in enumerate(image_patches):
    output_path = os.path.join(output_folder,f'slide_image_{cnt2}_{i}.png')
    if i==0:
        pass
    elif i%15==0:
        cnt2+=1
    
    cv2.imwrite(output_path,file)

In [ ]:
import numpy as np
file_list=glob.glob('./train_img/y/*.png')
output_folder='./data/y/'
    
image_width = 1024
image_height = 1024

window_size = 256

num_windows_x = (image_width - window_size) // window_size + 1
num_windows_y = (image_height - window_size) // window_size + 1

image_patches = []
for cnt,file in enumerate(file_list):
    image=cv2.imread(file)
    for i in range(num_windows_x):
        for j in range(num_windows_y):
            start_x = i * window_size
            start_y = j * window_size
            end_x = start_x + window_size
            end_y = start_y + window_size

            image_patch = image[start_y:end_y, start_x:end_x]
            image_patches.append(image_patch)

cnt2=0
for i,file in enumerate(image_patches):
    output_path = os.path.join(output_folder,f'slide_image_mask_{cnt2}_{i}.png')
    if i==0:
        pass
    elif i%15==0:
        cnt2+=1
    
    cv2.imwrite(output_path,file)

# 새로운 csv 생성


In [ ]:
lt_x = sorted(glob.glob('./data/x/*'))
lt_y = sorted(glob.glob('./data/y/*'))

new_df = pd.DataFrame()

for i in tqdm(range(len(lt_x))):
    img_id = lt_x[i].rsplit('/',1)[1][:-4]
    img_path = f'./data/x/{img_id}.png'
    mask_path = f'./data/y/{img_id}.png'
    a = pd.DataFrame([[img_id, img_path, mask_path]], columns=['img_id', 'img_path', 'mask_path'])
    new_df = pd.concat([new_df, a], ignore_index=True)
    
new_df.to_csv('./train_256_overlap.csv', index=False)